In [10]:
import pandas as pd
import numpy as np

In [30]:
class PHMToolWearDataset(object):
    # extract all data
    cache_dir_path = '.cache/'
    total_signal_num = 315
    # only 1,4 and 6 is correct
    sample_label = [1,4,6]
    sample_loc = 1

    def __init__(self):
        self.force_update = False

    @property
    def res_data_path(self):
        return '~/Downloads/c1/c%s_wear.csv' % (self.sample_loc)

    @property
    def get_res_data_in_numpy(self):
        # remove cache because it's not needed
        res_csv_data = self.get_res_data_by_pandas
        
        res_array = np.array([np.array(i).reshape(3) for i in res_csv_data.values])
        
        # np.save(storage_path, res_array)
        return res_array

    @property
    def get_res_data_by_pandas(self):
        return pd.read_csv(self.res_data_path, index_col='cut')

    @property
    def res_data_storage(self):
        return 'phm_tool_wear_data'

    @property
    def get_tool_wear_data(self):
        storage_path = self.cache_dir_path + self.res_data_storage
        self.sample_loc = 1
        y_dat = self.get_res_data_in_numpy
        for i in [4,6]:
            self.sample_loc = i
            y_dat = np.append(self.get_res_data_in_numpy, y_dat, axis=0)

       
        return y_dat



In [47]:
class RNNSeriesDataSet(object):

    def __init__(self,begin_timestep,end_timestep):
        a = PHMToolWearDataset()
        self.tool_wear_data = a.get_tool_wear_data
        
        self.max_tool_wear_data = np.max(self.tool_wear_data,axis=1)
       
        self.begin_timestep = begin_timestep
        self.end_timestep = end_timestep

    def get_tool_wear_data(self):
        return self.tool_wear_data

    def get_individual_tool_wear_batches(self,tool_wear_data):
        
        round_number = tool_wear_data.shape[0]
        
        begin_series = []
        end_series = []
        for start_index in range(self.begin_timestep,round_number-self.end_timestep):
            x = tool_wear_data[start_index-self.begin_timestep:start_index]
            y = tool_wear_data[start_index:start_index+self.end_timestep]
            
            begin_series.append(x)
            end_series.append(y)
        return begin_series,end_series

    def get_rnn_data(self):
        x,y = [],[]
        for i in range(3):
            ix,iy = self.get_individual_tool_wear_batches(self.max_tool_wear_data[i*315:(i+1)*315])
            x.extend(ix)
            y.extend(iy)
           
        dat_x,dat_y =  np.array(x),np.array(y)
        return dat_x.reshape((dat_x.shape[0],dat_x.shape[1],1)),dat_y.reshape((dat_y.shape[0],dat_y.shape[1],1))


In [48]:
data = RNNSeriesDataSet(10,20)
x,y = data.get_rnn_data()

# ---- shuffle -----
import random
# set random seeds so that we can get the same random data!
SEED = 12347
random.seed(SEED)
index = [i for i in range(len(y))]
random.shuffle(index)
train_y = y[index]
train_x = x[index]

print("Size :",train_x.shape,train_y.shape)

before: (945, 3)
after: (945,)
Size : (855, 10, 1) (855, 20, 1)


In [65]:
import tensorflow as tf
from tensorflow.keras.layers import GRU,Dense,RepeatVector
def build_stack_GRU(input_shape,output_length,output_dim):

    model = tf.keras.Sequential()
    model.add(GRU(64,input_shape=input_shape,return_sequences=False))
    #model.add(GRU(32,return_sequences=False))
    model.add(Dense(32))
    model.add(RepeatVector(output_length))
    model.add(GRU(32,return_sequences=False))
    #model.add(GRU(16,return_sequences=True))
    model.add(Dense(output_dim))

    model.compile(loss='mae', optimizer='adam', metrics=["mae", "mse"])
    return model

In [66]:
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
tb_cb = TensorBoard(log_dir="train_log")

In [67]:
model =  build_stack_GRU((10, 1), 20, 1)
model.fit(train_x,train_y,batch_size=16,epochs=2500,validation_split=0.2,callbacks=[tb_cb])

Train on 684 samples, validate on 171 samples
Epoch 1/2500
684/684 [==============================] - 4s 5ms/sample - loss: 114.2801 - mae: 114.2801 - mse: 14366.3330 - val_loss: 116.5075 - val_mae: 116.5074 - val_mse: 15178.7080
Epoch 2/2500
684/684 [==============================] - 1s 780us/sample - loss: 110.8150 - mae: 110.8150 - mse: 13576.4414 - val_loss: 114.8001 - val_mae: 114.8001 - val_mse: 14783.1934
Epoch 3/2500
684/684 [==============================] - 1s 788us/sample - loss: 109.2215 - mae: 109.2215 - mse: 13226.8906 - val_loss: 113.2603 - val_mae: 113.2603 - val_mse: 14432.0029
Epoch 4/2500
684/684 [==============================] - 1s 786us/sample - loss: 107.7047 - mae: 107.7047 - mse: 12897.0879 - val_loss: 111.7637 - val_mae: 111.7637 - val_mse: 14095.2002
Epoch 5/2500
684/684 [==============================] - 1s 790us/sample - loss: 106.2192 - mae: 106.2192 - mse: 12580.8057 - val_loss: 110.2877 - val_mae: 110.2878 - val_mse: 13767.4551
Epoch 6/2500
684/684 [====

Epoch 46/2500
684/684 [==============================] - 1s 808us/sample - loss: 48.3875 - mae: 48.3875 - mse: 3599.2822 - val_loss: 52.3739 - val_mae: 52.3739 - val_mse: 4321.6035
Epoch 47/2500
684/684 [==============================] - 1s 795us/sample - loss: 47.1697 - mae: 47.1697 - mse: 3472.2664 - val_loss: 51.1270 - val_mae: 51.1270 - val_mse: 4185.5879
Epoch 48/2500
684/684 [==============================] - 1s 785us/sample - loss: 45.9707 - mae: 45.9707 - mse: 3353.1008 - val_loss: 49.8891 - val_mae: 49.8891 - val_mse: 4052.6021
Epoch 49/2500
684/684 [==============================] - 1s 788us/sample - loss: 44.7811 - mae: 44.7811 - mse: 3235.7698 - val_loss: 48.6861 - val_mae: 48.6861 - val_mse: 3925.3115
Epoch 50/2500
684/684 [==============================] - 1s 785us/sample - loss: 43.6088 - mae: 43.6088 - mse: 3123.8242 - val_loss: 47.4896 - val_mae: 47.4896 - val_mse: 3800.7410
Epoch 51/2500
684/684 [==============================] - 1s 792us/sample - loss: 42.4465 - mae:

684/684 [==============================] - 1s 795us/sample - loss: 27.1130 - mae: 27.1130 - mse: 1520.5697 - val_loss: 30.5104 - val_mae: 30.5104 - val_mse: 1991.3021
Epoch 92/2500
684/684 [==============================] - 1s 808us/sample - loss: 27.0882 - mae: 27.0882 - mse: 1512.7592 - val_loss: 30.4715 - val_mae: 30.4715 - val_mse: 1979.9028
Epoch 93/2500
684/684 [==============================] - 1s 806us/sample - loss: 27.0632 - mae: 27.0632 - mse: 1505.4896 - val_loss: 30.4452 - val_mae: 30.4452 - val_mse: 1971.9750
Epoch 94/2500
684/684 [==============================] - 1s 801us/sample - loss: 27.0422 - mae: 27.0422 - mse: 1498.7521 - val_loss: 30.4129 - val_mae: 30.4129 - val_mse: 1962.0269
Epoch 95/2500
684/684 [==============================] - 1s 808us/sample - loss: 27.0216 - mae: 27.0216 - mse: 1492.5328 - val_loss: 30.3877 - val_mae: 30.3877 - val_mse: 1954.1251
Epoch 96/2500
684/684 [==============================] - 1s 808us/sample - loss: 27.0009 - mae: 27.0009 - mse

684/684 [==============================] - 1s 841us/sample - loss: 15.0294 - mae: 15.0294 - mse: 831.4263 - val_loss: 18.5729 - val_mae: 18.5729 - val_mse: 1166.6694
Epoch 137/2500
684/684 [==============================] - 1s 827us/sample - loss: 14.2366 - mae: 14.2366 - mse: 799.6932 - val_loss: 17.8810 - val_mae: 17.8810 - val_mse: 1128.9523
Epoch 138/2500
684/684 [==============================] - 1s 846us/sample - loss: 13.7299 - mae: 13.7299 - mse: 772.5759 - val_loss: 17.5710 - val_mae: 17.5710 - val_mse: 1094.2684
Epoch 139/2500
684/684 [==============================] - 1s 826us/sample - loss: 13.4244 - mae: 13.4244 - mse: 747.0544 - val_loss: 16.9700 - val_mae: 16.9700 - val_mse: 1057.8583
Epoch 140/2500
684/684 [==============================] - 1s 829us/sample - loss: 13.0424 - mae: 13.0424 - mse: 721.5464 - val_loss: 16.7059 - val_mae: 16.7059 - val_mse: 1024.6179
Epoch 141/2500
684/684 [==============================] - 1s 837us/sample - loss: 12.7356 - mae: 12.7356 - mse

Epoch 182/2500
684/684 [==============================] - 1s 811us/sample - loss: 7.0815 - mae: 7.0815 - mse: 277.3883 - val_loss: 9.1589 - val_mae: 9.1589 - val_mse: 386.3214
Epoch 183/2500
684/684 [==============================] - 1s 822us/sample - loss: 6.8425 - mae: 6.8425 - mse: 271.4530 - val_loss: 9.2623 - val_mae: 9.2623 - val_mse: 379.5103
Epoch 184/2500
684/684 [==============================] - 1s 806us/sample - loss: 6.8156 - mae: 6.8156 - mse: 266.4673 - val_loss: 9.1681 - val_mae: 9.1681 - val_mse: 373.6673
Epoch 185/2500
684/684 [==============================] - 1s 805us/sample - loss: 6.7641 - mae: 6.7641 - mse: 262.1577 - val_loss: 8.9099 - val_mae: 8.9099 - val_mse: 364.4803
Epoch 186/2500
684/684 [==============================] - 1s 803us/sample - loss: 6.6745 - mae: 6.6745 - mse: 256.9549 - val_loss: 8.6812 - val_mae: 8.6812 - val_mse: 356.4662
Epoch 187/2500
684/684 [==============================] - 1s 811us/sample - loss: 6.4883 - mae: 6.4883 - mse: 251.7390 -

...

Epoch 2296/2500
684/684 [==============================] - 1s 785us/sample - loss: 2.2677 - mae: 2.2677 - mse: 12.1015 - val_loss: 2.3676 - val_mae: 2.3676 - val_mse: 13.5905
Epoch 2297/2500
684/684 [==============================] - 1s 794us/sample - loss: 2.2676 - mae: 2.2676 - mse: 12.1385 - val_loss: 2.4142 - val_mae: 2.4142 - val_mse: 13.8549
Epoch 2298/2500
684/684 [==============================] - 1s 802us/sample - loss: 2.2785 - mae: 2.2785 - mse: 12.2334 - val_loss: 2.4031 - val_mae: 2.4031 - val_mse: 13.3690
Epoch 2299/2500
684/684 [==============================] - 1s 794us/sample - loss: 2.2633 - mae: 2.2633 - mse: 12.0603 - val_loss: 2.3861 - val_mae: 2.3861 - val_mse: 13.9476
Epoch 2300/2500
684/684 [==============================] - 1s 799us/sample - loss: 2.2716 - mae: 2.2716 - mse: 12.1181 - val_loss: 2.4557 - val_mae: 2.4557 - val_mse: 14.1001
Epoch 2301/2500
684/684 [==============================] - 1s 788us/sample - loss: 2.3259 - mae: 2.3259 - mse: 12.4704 - val_

Epoch 2343/2500
684/684 [==============================] - 1s 788us/sample - loss: 2.2677 - mae: 2.2677 - mse: 12.1176 - val_loss: 2.4065 - val_mae: 2.4065 - val_mse: 13.8482
Epoch 2344/2500
684/684 [==============================] - 1s 800us/sample - loss: 2.2961 - mae: 2.2961 - mse: 12.5038 - val_loss: 2.4069 - val_mae: 2.4069 - val_mse: 14.4298
Epoch 2345/2500
684/684 [==============================] - 1s 802us/sample - loss: 2.2805 - mae: 2.2805 - mse: 12.2604 - val_loss: 2.3930 - val_mae: 2.3930 - val_mse: 13.6705
Epoch 2346/2500
684/684 [==============================] - 1s 798us/sample - loss: 2.2734 - mae: 2.2734 - mse: 12.1703 - val_loss: 2.3594 - val_mae: 2.3594 - val_mse: 13.3819
Epoch 2347/2500
684/684 [==============================] - 1s 794us/sample - loss: 2.2772 - mae: 2.2772 - mse: 12.1928 - val_loss: 2.3827 - val_mae: 2.3827 - val_mse: 13.4751
Epoch 2348/2500
684/684 [==============================] - 1s 799us/sample - loss: 2.2899 - mae: 2.2899 - mse: 12.3036 - val_

Epoch 2390/2500
684/684 [==============================] - 1s 791us/sample - loss: 2.2708 - mae: 2.2708 - mse: 12.1014 - val_loss: 2.4125 - val_mae: 2.4125 - val_mse: 13.2923
Epoch 2391/2500
684/684 [==============================] - 1s 790us/sample - loss: 2.2586 - mae: 2.2586 - mse: 12.0534 - val_loss: 2.4032 - val_mae: 2.4032 - val_mse: 13.4287
Epoch 2392/2500
684/684 [==============================] - 1s 790us/sample - loss: 2.2626 - mae: 2.2626 - mse: 12.0694 - val_loss: 2.3829 - val_mae: 2.3829 - val_mse: 13.4467
Epoch 2393/2500
684/684 [==============================] - 1s 791us/sample - loss: 2.3642 - mae: 2.3642 - mse: 12.6792 - val_loss: 2.5602 - val_mae: 2.5602 - val_mse: 15.1449
Epoch 2394/2500
684/684 [==============================] - 1s 797us/sample - loss: 2.3072 - mae: 2.3072 - mse: 12.4196 - val_loss: 2.3857 - val_mae: 2.3857 - val_mse: 13.6285
Epoch 2395/2500
684/684 [==============================] - 1s 797us/sample - loss: 2.2618 - mae: 2.2618 - mse: 12.1462 - val_

Epoch 2437/2500
684/684 [==============================] - 1s 794us/sample - loss: 2.2649 - mae: 2.2649 - mse: 12.1073 - val_loss: 2.4146 - val_mae: 2.4146 - val_mse: 13.3276
Epoch 2438/2500
684/684 [==============================] - 1s 797us/sample - loss: 2.2727 - mae: 2.2727 - mse: 12.1467 - val_loss: 2.3479 - val_mae: 2.3479 - val_mse: 13.3346
Epoch 2439/2500
684/684 [==============================] - 1s 801us/sample - loss: 2.2914 - mae: 2.2914 - mse: 12.2016 - val_loss: 2.4275 - val_mae: 2.4275 - val_mse: 13.9043
Epoch 2440/2500
684/684 [==============================] - 1s 793us/sample - loss: 2.2638 - mae: 2.2638 - mse: 12.0283 - val_loss: 2.3939 - val_mae: 2.3939 - val_mse: 13.7776
Epoch 2441/2500
684/684 [==============================] - 1s 792us/sample - loss: 2.2646 - mae: 2.2646 - mse: 12.1526 - val_loss: 2.3691 - val_mae: 2.3691 - val_mse: 13.3939
Epoch 2442/2500
684/684 [==============================] - 1s 794us/sample - loss: 2.2969 - mae: 2.2969 - mse: 12.3392 - val_

Epoch 2484/2500
684/684 [==============================] - 1s 799us/sample - loss: 2.2691 - mae: 2.2691 - mse: 12.1778 - val_loss: 2.3588 - val_mae: 2.3588 - val_mse: 13.3041
Epoch 2485/2500
684/684 [==============================] - 1s 789us/sample - loss: 2.2841 - mae: 2.2841 - mse: 12.3024 - val_loss: 2.3920 - val_mae: 2.3920 - val_mse: 13.4576
Epoch 2486/2500
684/684 [==============================] - 1s 796us/sample - loss: 2.2657 - mae: 2.2657 - mse: 12.0494 - val_loss: 2.3576 - val_mae: 2.3576 - val_mse: 13.4486
Epoch 2487/2500
684/684 [==============================] - 1s 799us/sample - loss: 2.2855 - mae: 2.2855 - mse: 12.1584 - val_loss: 2.4023 - val_mae: 2.4023 - val_mse: 13.6131
Epoch 2488/2500
684/684 [==============================] - 1s 798us/sample - loss: 2.2976 - mae: 2.2976 - mse: 12.2662 - val_loss: 2.3850 - val_mae: 2.3850 - val_mse: 13.9784
Epoch 2489/2500
684/684 [==============================] - 1s 790us/sample - loss: 2.2895 - mae: 2.2895 - mse: 12.2147 - val_

In [68]:
!tensorboard --logdir=train_log

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.0.2 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [70]:
print(np.mean(train_x))

114.03485365397076
